<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Forward-Propagation" data-toc-modified-id="Forward-Propagation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Forward Propagation</a></span><ul class="toc-item"><li><span><a href="#Update-State" data-toc-modified-id="Update-State-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Update State</a></span></li><li><span><a href="#Forward-States" data-toc-modified-id="Forward-States-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Forward States</a></span></li><li><span><a href="#Cost/Loss-Function" data-toc-modified-id="Cost/Loss-Function-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Cost/Loss Function</a></span></li></ul></li><li><span><a href="#Backward-Propagation" data-toc-modified-id="Backward-Propagation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Backward Propagation</a></span><ul class="toc-item"><li><span><a href="#Cost-Gradient" data-toc-modified-id="Cost-Gradient-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Cost Gradient</a></span></li><li><span><a href="#Backward" data-toc-modified-id="Backward-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Backward</a></span></li><li><span><a href="#gradient-checking" data-toc-modified-id="gradient-checking-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>gradient checking</a></span></li></ul></li><li><span><a href="#Optimizer" data-toc-modified-id="Optimizer-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Optimizer</a></span></li><li><span><a href="#Testing" data-toc-modified-id="Testing-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Testing</a></span></li></ul></div>

# RNN From Scrath

What better way to learn RNN than to write it from scratch? 

Goal: Build an RNN that given a sequence of binary numbers, returns the sum of the sequence. For example, given [1,1,0] -> returns 2

The original code is from the link below but I made some changes. 

Credit: http://peterroelants.github.io/posts/rnn_implementation_part01/#Linear-recurrent-neural-network

In [2]:
import numpy as np
# python version 
import sys
print('python version:', sys.version[:3])

python version: 3.6


<caption><center> **RNN**  </center></caption>
<img src="SimpleRNN01.png" border="4" style="width:700px">

## Setup

In [3]:
# Create dataset
m = 20 # number of samples 
sequence_len = 10
# Create the sequences
X = np.zeros((m, sequence_len))
for row_idx in range(m):
    X[row_idx,:] = np.around(np.random.rand(sequence_len)).astype(int)
# Create the targets for each sequence
t = np.sum(X, axis=1) # = y

In [4]:
X.shape # 20 examples, 10 sequences each 

(20, 10)

In [5]:
# print input
X

array([[0., 0., 0., 1., 1., 0., 1., 1., 1., 1.],
       [1., 1., 1., 0., 0., 0., 1., 1., 1., 1.],
       [0., 0., 1., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 1., 0., 1., 1., 0.],
       [1., 0., 0., 1., 1., 0., 1., 1., 1., 1.],
       [1., 0., 0., 0., 1., 1., 1., 1., 1., 0.],
       [1., 0., 1., 0., 0., 1., 0., 1., 1., 0.],
       [1., 1., 1., 0., 1., 0., 0., 1., 0., 0.],
       [1., 1., 1., 1., 0., 0., 1., 0., 0., 1.],
       [1., 1., 1., 0., 0., 0., 1., 1., 0., 0.],
       [0., 1., 0., 1., 1., 0., 1., 0., 1., 0.],
       [0., 1., 0., 1., 0., 1., 1., 1., 0., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
       [0., 1., 1., 0., 0., 1., 0., 0., 1., 1.],
       [1., 1., 1., 0., 1., 1., 1., 0., 0., 0.],
       [1., 1., 1., 0., 1., 0., 1., 0., 1., 0.],
       [0., 1., 1., 1., 0., 1., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 1., 0.],
       [1., 0., 1., 1., 0., 1., 1., 1., 0., 1.]])

In [6]:
# print output 
t # y 

array([6., 7., 2., 4., 5., 7., 6., 5., 5., 6., 5., 5., 5., 7., 5., 6., 6.,
       7., 2., 7.])

## Forward Propagation

### Update State

Compute activation function

In [7]:
# Define the forward step functions
def update_state(xk, sk, wx, wRec): # (change compute activation)
    """
    Compute activation at time step k (change to t) by using information from precious state(activation) 
    and current input xk (change to x_t). 
    Normally, there would be a bais term and tanh activation function. 
    
    k = current time step (change to t)
    sk = previous state (change to s_prev or a_prev for previous activation)  
    xk = current input (change to xt)   
    xw = input weights (w_x)
    wRec = recursive weights (w_a) 

    Returns the activation of current time step t 
    """
    
    # current activation (a_t) = input * input weight + previous activation * activation weight 
    # a_t = x_t * w_x + a_prev * x_a
    current_state = xk * wx + sk * wRec 
    
    return current_state

### Forward States

In [8]:
def forward_states(X, wx, wRec): # (change to forward_propagration)
    """
    Unfold the network and compute all state activations given the input X,
    and input weights (wx or w_x) and recursive weights (wRec or x_a).
    
    Return the activations in a matrix, the last column S[:,-1] contains the
    final activations.
    """
    
    # Initialise the matrix that holds all states for all input sequences.
    # The initial state s0 is set to 0.
    S = np.zeros((X.shape[0], X.shape[1]+1)) # S.shape: (20, 11) 
     
    # Use the recurrence relation defined by update_state to update the states trough time.
    # go through the sequence (10 length)
    for k in range(0, X.shape[1]): #(change k to t)
        # S[k] = S[k-1] * wRec + X[k] * wx
        '''
        X[:,k] -> grabs k-th column, meaning grab every example at time k (or t)
        S[:,k] -> grabbing the previous state, it will be all 0 for the first loop iteration 
        '''
        S[:,k+1] = update_state(X[:,k], S[:,k], wx, wRec)
        
    return S

### Cost/Loss Function

In [9]:
def cost(y_hat, y): 
    """
    y = t = target (change to target or y or the true label) 
    y_hat = output = y_hat (change to y_hat or last activation unit or output)
    
    Return the MSE between the targets t and the outputs y.
    
    m = number of samples 
    """
    
    return ((y - y_hat)**2).sum() / m 

## Backward Propagation

### Cost Gradient

In [34]:
def output_gradient(y_hat, y): # change name 
    """
    y = t = target (change to target or y or the true label) 
    y_hat = output = y_hat (change to y_hat or last activation unit or output)
    
    return -> derivative of MSE = ((t - y)**2).sum() / m
    
    Compute the gradient of the MSE cost function with respect to the output y_hat or last activation.
    Computes the last layer backpropagation 
    """
    return 2.0 * (y_hat - y) / m

### Backward 

In [17]:
def backward_gradient(X, S, grad_out, wRec):
    """    
    Backpropagate the gradient computed at the output (grad_out) through the network.
    Accumulate the parameter gradients for wX and wRec by for each layer by addition.
    
    Return the parameter gradients as a tuple, and the gradients at the output of each layer.
    
    Formula: 
        dL/dw_x = 2/m * (a_t - y) * w_a^[] * x_t 
        dL/dw_a = 2/m * (a_t - y) * w_a^[] * a_(t-1) 
    
    """
    
    # Initialise the array that stores the gradients of the cost with respect to the states.
    grad_over_time = np.zeros((X.shape[0], X.shape[1]+1))
    
    # the first starts from the last column 
    grad_over_time[:,-1] = grad_out
    
    # Set the gradient accumulations to 0 
    wx_grad = 0 # w_x_grad 
    wRec_grad = 0 # w_a_grad 
    
    # loop through each time step (or the length of the sequence)
    for k in range(X.shape[1], 0, -1): # X.shape[1] = 10 
        
        # Compute the parameter gradients and accumulate the results.
        wx_grad += np.sum(grad_over_time[:,k] * X[:,k-1])
        wRec_grad += np.sum(grad_over_time[:,k] * S[:,k-1])
        
        # Compute the gradient at the output of the previous layer 
        grad_over_time[:,k-1] = grad_over_time[:,k] * wRec 
        
    return (wx_grad, wRec_grad), grad_over_time

### gradient checking

In [21]:
# Perform gradient checking
# Set the weight parameters used during gradient checking
params = [1.2, 1.2]  # [wx, wRec]

# Set the small change to compute the numerical gradient
eps = 1e-7

# Forward propagation 
S = forward_states(X, params[0], params[1])
print(S.shape)

grad_out = output_gradient(S[:,-1], t) # t = target values  
print(grad_out.shape)
      
backprop_grads, grad_over_time = backward_gradient(X, S, grad_out, params[1]) # why pass in just second param. wRec? 
print(backprop_grads, grad_over_time.shape)

(20, 11)
(20,)
(-375.6854816395034, -2329.373310054043) (20, 11)


In [28]:
# Perform gradient checking
# Set the weight parameters used during gradient checking
params = [1.2, 1.2]  # [wx, wRec]
# Set the small change to compute the numerical gradient
eps = 1e-7
# Compute the backprop gradients
S = forward_states(X, params[0], params[1])
grad_out = output_gradient(S[:,-1], t)
backprop_grads, grad_over_time = backward_gradient(X, S, grad_out, params[1])
# Compute the numerical gradient for each parameter in the layer
for p_idx, _ in enumerate(params):
    grad_backprop = backprop_grads[p_idx]
    # + eps
    params[p_idx] += eps
    plus_cost = cost(forward_states(X, params[0], params[1])[:,-1], t)
    # - eps
    params[p_idx] -= 2 * eps
    min_cost = cost(forward_states(X, params[0], params[1])[:,-1], t)
    # reset param value
    params[p_idx] += eps
    # calculate numerical gradient
    grad_num = (plus_cost - min_cost) / (2*eps)
    # Raise error if the numerical grade is not close to the backprop gradient
    if not np.isclose(grad_num, grad_backprop):
        print ('Numerical gradient of {:.6f} is not close to the backpropagation gradient of {:.6f}!'.format(float(grad_num), float(grad_backprop)))
print('No gradient errors found')

No gradient errors found


## Optimizer

In [29]:
# Define Rprop optimisation function
def update_rprop(X, t, W, W_prev_sign, W_delta, eta_p, eta_n):
    """
    Update Rprop values in one iteration.
    X: input data.
    t: targets.
    W: Current weight parameters.
    W_prev_sign: Previous sign of the W gradient.
    W_delta: Rprop update values (Delta).
    eta_p, eta_n: Rprop hyperparameters.
    """
    # Perform forward and backward pass to get the gradients
    S = forward_states(X, W[0], W[1])
    grad_out = output_gradient(S[:,-1], t)
    W_grads, _ = backward_gradient(X, S, grad_out, W[1])
    W_sign = np.sign(W_grads)  # Sign of new gradient
    # Update the Delta (update value) for each weight parameter seperately
    for i, _ in enumerate(W):
        if W_sign[i] == W_prev_sign[i]:
            W_delta[i] *= eta_p
        else:
            W_delta[i] *= eta_n
    return W_delta, W_sign

In [44]:
# Perform Rprop optimisation

# Set hyperparameters
eta_p = 1.2
eta_n = 0.5

# Set initial parameters
#W = [-1.5, 2]  # [wx, wRec]
W = [-10, 7]  # [wx, wRec]
W_delta = [0.001, 0.001]  # Update values (Delta) for W
W_sign = [0, 0]  # Previous sign of W

ls_of_ws = [(W[0], W[1])]  # List of weights to plot
# Iterate over 500 iterations
for i in range(10000):
    # Get the update values and sign of the last gradient
    W_delta, W_sign = update_rprop(X, t, W, W_sign, W_delta, eta_p, eta_n)
    # Update each weight parameter seperately
    for i, _ in enumerate(W):
        W[i] -= W_sign[i] * W_delta[i]
    ls_of_ws.append((W[0], W[1]))  # Add weights to list to plot

print('Final weights are: wx = {0},  wRec = {1}'.format(W[0], W[1]))

Final weights are: wx = 0.9999999999999711,  wRec = 1.0000000000000062


## Testing

In [32]:
test_inpt = np.asmatrix([[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1]])
test_outpt = forward_states(test_inpt, W[0], W[1])[:,-1]
print ('Target output: {:d} vs Model output: {:.2f}'.format(test_inpt.sum(), test_outpt[0]))

Target output: 5 vs Model output: 5.00


In [33]:
test_inpt = np.asmatrix([[0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1]])
test_outpt = forward_states(test_inpt, W[0], W[1])[:,-1]
print ('Target output: {:d} vs Model output: {:.2f}'.format(test_inpt.sum(), test_outpt[0]))

Target output: 7 vs Model output: 7.00


In [35]:
test_inpt = np.asmatrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]])
test_outpt = forward_states(test_inpt, W[0], W[1])[:,-1]
print ('Target output: {:d} vs Model output: {:.2f}'.format(test_inpt.sum(), test_outpt[0]))

Target output: 2 vs Model output: 2.00
